## WRA station processing
This notebook analyses the Excel file provided to us by the WRA staff. Multiple stations had some inconsistent formatting of the sheets, so this notebook first resolves the format of the Excel format. The discharge and water level measurements are then matched together based on the time of their recording. 

The next step would be to calculate the discharge for each water level based on the available datapoints. However, the data that is available for stations is rather limited, so some more measurements may need to be done first. 

After processing each station is saved as a .csv file in the directory with the name of the station as the file name for easy identification.

In [2]:
import numpy as np
import pandas as pd
import pathlib
import matplotlib as mpl
import matplotlib.pyplot as plt
import sys
import os.path
from matplotlib.ticker import MaxNLocator
import warnings

In [23]:
# If errors occur here please refer to the readme file or to the file_imports.py folders. 

cwd = pathlib.Path().resolve()
src = cwd.parent
data = src.parent.parent.parent
root = src.parent
sys.path.append(str(src))
sys.path.append(str(root))
from utils.file_imports import *


data_paths = file_paths(root, WRA = True)
waterlevel_files = data_paths[1]

path = os.path.join(waterlevel_files, 'Tana_data.xlsx')
sheet = pd.ExcelFile(path)
sheetlist = sheet.sheet_names

The first entry is pointing to /Users/matskerver/Documents/data_tana/WRA/Garissa_station, the second one to /Users/matskerver/Documents/data_tana/WRA/other_stations and


In [24]:
# Function to find the name of the station, as this is not contained in the sheet name but rather in the first
# column of the excel sheet itself. 

def get_station_name(df):
    column_index = 1
    column_names = df.columns.tolist()
    column_name = column_names[column_index]
    parts = column_name.split('-')
    extracted_name = parts[0].strip()  
    return extracted_name

In [18]:
# Save all the station names in a single array for naming of the .csv files later.

names_stations = []
for name in sheetlist:    
    df = sheet.parse(name)
    names_stations.append(get_station_name(df))

['4BD01 MATHIOYA', '4BC04 RWAMUTHAMBI', '4BE10 TANA RUKANGA', '4BB01 RAGATI', '4AC03 SAGANA', '4AA05 SAGANA', '4AB06 AMBONI', '4AC04 NEW CHANIA', '4AA04 NAIROBI', '4AA02 THEGO', '4AA01 SAGANA', '4AB01 MURINGATO', '4BC05 RWAMUTHAMBI', '4BE04 KAYAHWE', '4CB04 THIKA', '4BF01 SABA SABA', '4AD01 GURA', '4BD07 MATHIOYA', '4BE01 MARAGUA', '4CA03 CHANIA', '4CA19 KARIMINU', '4DA14 KAMWETI', '4DA13 KIRINGA', '4DB05 NYAMINDI', '4DA10 THIBA', '4BC05 RWAMUTHAMBI', '4DD02 THIBA', '4EA03 KITHINO RIVER']


In [25]:
# Loop through all the sheets to format them properly and save them to a .csv file.

for i in range(0, len(sheetlist)):
    
    # Read the excel file for the current sheet and create a dataframe that will later be converted to .csv
    df = pd.read_excel(path, sheet_name=sheetlist[i])
    columns_to_drop = [2, 3, 4]  
    df = df.drop(df.columns[columns_to_drop], axis=1)
    df.columns = [
        "waterlevel(m)" if i == 1 else
        "Time_Q" if i == 2 else
        "discharge(m3/s)" if i == 3 else col
        for i, col in enumerate(df.columns)
    ]
    
    # These lines resolve the inconsistent datetime formatting of the dataset. 
    df['Time'] = pd.to_datetime(df['Time'], format='mixed')
    df['Time_Q'] = pd.to_datetime(df['Time_Q'], format='mixed')

    # Drop empty cells out of the dataframe
    df_level = df.iloc[:,0:2].dropna()
    df_discharge = df.iloc[:,2:4].dropna()
    df_discharge = df_discharge.rename(columns={"Time_Q": "Time"})

    # Match the discharge data with the water level data where available. Tolerance to be changed based on 
    # nature of river/stream and the required accuracy.
    merged_dataframe = pd.merge_asof(df_level, df_discharge, on="Time", tolerance=pd.Timedelta("1days"))
    merged_dataframe.head()
    
    #Save each file to a .csv file
    save_path = os.path.join(waterlevel_files, f"Tana_station_{names_stations[i]}.csv")
    merged_dataframe.to_csv(save_path, index = False)